In [3]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from tkinter import *
import tkinter as tk
import PIL
import PIL.Image
from PIL import ImageTk
from tkinter import messagebox
from tkinter import scrolledtext

In [4]:
df = pd.read_csv("Dataset_all.csv")

le = preprocessing.LabelEncoder()
df['Lattice Type Encoded'] = le.fit_transform(df['Lattice Type'])

scaler = StandardScaler()
columns_to_normalize = ['X', 'Y', 'Z', 'Thickness', 'Force (N)', 'Strain (mm)']

std_values = np.std(df[['X', 'Y', 'Z', 'Thickness']], axis=0)
mean_values = np.mean(df[['X', 'Y', 'Z', 'Thickness']], axis=0)

std_force = np.std(df['Force (N)'], axis=0)
mean_force = np.mean(df['Force (N)'], axis=0)

std_strain = np.std(df['Strain (mm)'], axis=0)
mean_strain = np.mean(df['Strain (mm)'], axis=0)

for column in columns_to_normalize:
    column_data = np.array(df[column]).reshape(-1, 1)
    df[column] = scaler.fit_transform(column_data).flatten()

x_data = df[['Lattice Type Encoded', 'Force (N)', 'Strain (mm)']]
y_data = df[['X', 'Y', 'Z', 'Thickness']]

model = RandomForestRegressor(max_depth=20, n_estimators=200)

model.fit(x_data, y_data)

RandomForestRegressor(max_depth=20, n_estimators=200)

In [5]:
app = tk.Tk()
app.title("Predicting Lattice Structure")
app.geometry("1000x800")
app.minsize(200, 300)
app.iconbitmap("Tkinter\logo.ico")
app.configure(bg='lightskyblue2')


def values_between(start, end, num_values):
    if num_values <= 0:
        return []

    step = (end - start) / (num_values + 1)
    values = [start + i * step for i in range(1, num_values + 1)]
    return values

def update_result():
    lattice_type = lattice_type_var.get()
    try:
        force1 = float(entry_force1.get())
        strain1 = float(entry_strain1.get())
        if not (0 <= force1 <= 600) or not (0 <= strain1 <= 14.5):
            raise ValueError("Force and Strain values must be within specified ranges.")
        force2 = float(entry_force2.get())
        strain2 = float(entry_strain2.get())
        if not (0 <= force2 <= 600) or not (0 <= strain2 <= 14.5):
            raise ValueError("Force and Strain values must be within specified ranges.")

        lattice_type_encoded = le.transform([lattice_type])[0]
        
        results_text = ""
        
        intermediate_values = []
        
        for force, strain, set_num in [(force1, strain1, 1), (force2, strain2, 2)]:
            force_normalized = (force - mean_force) / std_force
            strain_normalized = (strain - mean_strain) / std_strain

            input_data = [[lattice_type_encoded, force_normalized, strain_normalized]]
            predictions = model.predict(input_data)[0]

            for i, column in enumerate(['X', 'Y', 'Z', 'Thickness']):
                predictions[i] = predictions[i] * std_values[i] + mean_values[i]

            results_text += f"\nUnit Cell Geometries for Zone {set_num}: "
            results_text += f"X: {predictions[0]:.2f} mm, Y: {predictions[1]:.2f} mm, Z: {predictions[2]:.2f} mm, Thickness: {predictions[3]:.2f} mm\n"
            
            if set_num == 1:
                predictions1 = predictions
            else:
                predictions2 = predictions
                
                num_intermediate = int(entry_num_intermediate.get())
                
                intermediate_values.append([values_between(p1, p2, num_intermediate) for p1, p2 in zip(predictions1, predictions2)])
                
        results_text += "\n\nUnit Cell Geometries for Zones between Zone 1 and Zone 2:\n\n"
        for i, column in enumerate(['X', 'Y', 'Z', 'Thickness']):
            results_text += f"{column}: "
            for j, values in enumerate(intermediate_values):
                results_text += ", ".join([f"{v:.2f}" for v in values[i]])
                if j < len(intermediate_values) - 1:
                    results_text += ", "
                else:
                    results_text += ".\n"
                
        result_text.config(state=tk.NORMAL)
        result_text.delete('1.0', tk.END)
        result_text.insert(tk.END, results_text)
        result_text.config(state=tk.DISABLED)
        
    except ValueError as e:
        messagebox.showerror("Input Error", str(e))


images = {
    'Diamond': "Tkinter\Diamond.png",  
    'Gyroid': "Tkinter\Gyroid.png",     
    'Lidinoid': "Tkinter\Lidinoid.png", 
    'Schwarz': "Tkinter\Schwarz.png",   
    'SplitP': "Tkinter\SplitP.png"      
} 

def update_image(*args):
    lattice_structure = lattice_type_var.get()
    image_path = images[lattice_structure]
    img = PIL.Image.open(image_path)
    img = img.resize((120, 120), PIL.Image.LANCZOS)
    img = ImageTk.PhotoImage(img)
    image_label.config(image=img)
    image_label.image = img
        
def show_limits():
    limits_text = "Force Range: 0 to 600 N\nStrain Range: 0 to 14.5 mm"
    messagebox.showinfo("Input Limits", limits_text)

def create_tooltip(widget, text):
    widget.bind("<Enter>", lambda event: show_tooltip(widget, text))
    widget.bind("<Leave>", lambda event: hide_tooltip())

def show_tooltip(widget, text):
    x, y, _, _ = widget.bbox("insert")
    x += widget.winfo_rootx() + 25
    y += widget.winfo_rooty() + 25
    
    tooltip.geometry("+%d+%d" % (x, y))
    tooltip.wm_overrideredirect(True)
    tooltip.wm_geometry("+" + str(x) + "+" + str(y))
    tooltip_label.config(text=text)
    tooltip.deiconify()

def hide_tooltip():
    tooltip.withdraw()
    
tooltip = tk.Toplevel(app)
tooltip.withdraw()
tooltip_label = tk.Label(tooltip, text="", justify='left', background='#ffffe0', relief='solid', borderwidth=1,
                         font=('Helvetica', 10, 'normal'))
tooltip_label.pack(ipadx=5)

        
header = Label(app, text="Prediction of Lattice Structures Using Force & Strain",
               font=('Helvetica', 24, 'bold'), bg='lightskyblue2', borderwidth=5, foreground='dodgerblue4')
header.pack(side=TOP)

header2 = Label(app, text="Please Enter Your Inputs!",
                font=('Helvetica', 16, 'bold'), bg='lightskyblue2', borderwidth=7, foreground='crimson')
header2.pack()

frame = Frame(app, bg='lightskyblue2')
frame.pack()


label_lattice_type = tk.Label(frame, text="Lattice Type:",
                              font=('Helvetica', 12, 'bold'), bg='lightskyblue2', borderwidth=5)
label_lattice_type.grid(row=0, column=0)

lattice_type_var = tk.StringVar()
lattice_type_var.set('Diamond')
lattice_type_var.trace_add("write", update_image)

lattice_types = ['Diamond', 'Gyroid', 'Lidinoid', 'Schwarz', 'SplitP']
lattice_type_menu = tk.OptionMenu(frame, lattice_type_var, *lattice_types)
lattice_type_menu.grid(row=0, column=1)
lattice_type_menu.configure(font=('Helvetica', 11), bg='lightskyblue1', activebackground='lightskyblue3', borderwidth=2)
lattice_type_menu["menu"].config(font=('Helvetica', 11), bg='lightskyblue3')

image_path = images[lattice_type_var.get()]
default_img = PIL.Image.open(image_path)
default_img = default_img.resize((120, 120), PIL.Image.LANCZOS)
default_img = ImageTk.PhotoImage(default_img)
image_label = tk.Label(frame, text="Related Image:", image=default_img, bg='lightskyblue2')
image_label.grid(row=0, column=2)



label_force1_zone = tk.Label(frame, text="Zone 1",
                             font=('Helvetica', 14, 'bold'), bg='lightskyblue2', borderwidth=0)
label_force1_zone.grid(row=1, column=1, sticky="n")
label_force1 = tk.Label(frame, text="Force (N):",
                       font=('Helvetica', 12, 'bold'), bg='lightskyblue2', borderwidth=7)
label_force1.grid(row=2, column=0)
entry_force1 = tk.Entry(frame, font=('Helvetica', 12), borderwidth=2)
entry_force1.grid(row=2, column=1)

help_icon_image = tk.PhotoImage(file="Tkinter\help_icon.png") 
help_icon = tk.Label(frame, image=help_icon_image, bg='lightskyblue2', cursor="hand2")
help_icon.grid(row=2, column=2)
create_tooltip(help_icon, "Click to view input limits")
help_icon.bind("<Button-1>", lambda event: show_limits())

label_strain1 = tk.Label(frame, text="Strain (mm):",
                        font=('Helvetica', 12, 'bold'), bg='lightskyblue2', borderwidth=7)
label_strain1.grid(row=3, column=0)
entry_strain1 = tk.Entry(frame, font=('Helvetica', 12), borderwidth=2)
entry_strain1.grid(row=3, column=1)


label_force2_zone = tk.Label(frame, text="Zone 2",
                             font=('Helvetica', 14, 'bold'), bg='lightskyblue2', borderwidth=0)
label_force2_zone.grid(row=4, column=1, sticky="n")



label_force2 = tk.Label(frame, text="Force (N):",
                       font=('Helvetica', 12, 'bold'), bg='lightskyblue2', borderwidth=7)
label_force2.grid(row=5, column=0)
entry_force2 = tk.Entry(frame, font=('Helvetica', 12), borderwidth=2)
entry_force2.grid(row=5, column=1)

label_strain2 = tk.Label(frame, text="Strain (mm):",
                        font=('Helvetica', 12, 'bold'), bg='lightskyblue2', borderwidth=7)
label_strain2.grid(row=6, column=0)
entry_strain2 = tk.Entry(frame, font=('Helvetica', 12), borderwidth=2)
entry_strain2.grid(row=6, column=1)


label_intermediate_zone = tk.Label(frame, text="Intermediate Zones",
                             font=('Helvetica', 14, 'bold'), bg='lightskyblue2', borderwidth=0)
label_intermediate_zone.grid(row=7, column=1, sticky="n")

label_num_intermediate = tk.Label(frame, text="Number of Intermediate Zones:",
                                  font=('Helvetica', 12, 'bold'), bg='lightskyblue2', borderwidth=7)
label_num_intermediate.grid(row=8, column=0)
entry_num_intermediate = tk.Entry(frame, font=('Helvetica', 12), borderwidth=2)
entry_num_intermediate.grid(row=8, column=1)


predict_button = tk.Button(app, text="Predict", command=update_result,
                           font=('Helvetica', 14, 'bold'), borderwidth=5)
predict_button.pack()


result_text = tk.Text(app, wrap="none", font=('Helvetica', 11, 'bold'), width=80, height=20)

horizontal_scrollbar = tk.Scrollbar(app, orient="horizontal", command=result_text.xview)
result_text.configure(xscrollcommand=horizontal_scrollbar.set, bg='lightskyblue2')

horizontal_scrollbar.pack(side="bottom", fill="x")
result_text.pack(expand=True, fill="both")

app.bind('<Return>', lambda event=None: update_result())
app.mainloop()

D:\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
D:\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
